In [1]:
ORION_SERVER_PRIVATE=192.168.4.230
ORION_SERVER_PUBLIC=84.89.60.4
ORION_PORT=80

In [2]:
ORION_SERVER=$ORION_SERVER_PRIVATE

## Installation / configuration

### Connectivity test

In [9]:
curl http://$ORION_SERVER:$ORION_PORT/version

{
"orion" : {
  "version" : "1.11.0-next",
  "uptime" : "0 d, 1 h, 6 m, 41 s",
  "git_hash" : "781d3b621e88a5124be8c6ae3f57a88a3dcc160a",
  "compile_time" : "Wed Feb 14 11:09:08 UTC 2018",
  "compiled_by" : "root",
  "compiled_in" : "88eb02e35773",
  "release_date" : "Wed Feb 14 11:09:08 UTC 2018",
  "doc" : "https://fiware-orion.readthedocs.org/en/master/"
}
}



### Register entity

Each entity must have an unique identifier!!

Remember to define your related type, would be used to filter all available sensors

In [4]:
MY_UNIQUE_ID="bressol"
MY_TYPE="bebe"

In [5]:
curl $ORION_SERVER:$ORION_PORT/v2/entities \
   -X POST -s -S --header  'Content-Type: application/json' -d @- <<EOF
{
  "id": "$MY_UNIQUE_ID",
  "type": "$MY_TYPE",
  "description": {
    "value": "Bressol automàtic per bebes",
    "type": "String"
  },
  "estat": {
    "value": "on",
    "type": "String"
  }
}
EOF

### Query this entity

Based on our type, fetch it from the server

In [70]:
curl $ORION_SERVER:$ORION_PORT/v2/entities?type=$MY_TYPE

[{"id":"my_entityasdada","type":"my_type","city_location":{"type":"City","value":"Madrid","metadata":{}},"temperaturetttt":{"type":"Number","value":23.8,"metadata":{}}},{"id":"my_entityasdaaada","type":"my_type","description":{"type":"City","value":"Madrid","metadata":{}},"temperaturetttt":{"type":"Number","value":23.8,"metadata":{}}},{"id":"my_entityasdaaaada","type":"my_type","description":{"type":"String","value":"Summary of this entity...","metadata":{}},"temperaturetttt":{"type":"Number","value":23.8,"metadata":{}}},{"id":"my_entityasdaxaaada","type":"my_type","description":{"type":"String","value":"Summary of this entity...","metadata":{}},"var1":{"type":"Float","value":23.8,"metadata":{}}}]


### Search by id

Search entities based on their ID

In [81]:
curl $ORION_SERVER:$ORION_PORT/v2/entities/$MY_UNIQUE_ID -s -S | python -m json.tool

{
    "id": "my_entityasdaxaaada",
    "type": "my_type",
    "description": {
        "type": "String",
        "value": "Summary of this entity...",
        "metadata": {}
    },
    "var1": {
        "type": "Float",
        "value": 23.8,
        "metadata": {}
    }
}



### List entities

Reach all available entities

In [82]:
curl $ORION_SERVER:$ORION_PORT/v2/entities?type=$MY_TYPE -s -S | python -m json.tool

[
    {
        "id": "my_entityasdada",
        "type": "my_type",
        "city_location": {
            "type": "City",
            "value": "Madrid",
            "metadata": {}
        },
        "temperaturetttt": {
            "type": "Number",
            "value": 23.8,
            "metadata": {}
        }
    },
    {
        "id": "my_entityasdaaada",
        "type": "my_type",
        "description": {
            "type": "City",
            "value": "Madrid",
            "metadata": {}
        },
        "temperaturetttt": {
            "type": "Number",
            "value": 23.8,
            "metadata": {}
        }
    },
    {
        "id": "my_entityasdaaaada",
        "type": "my_type",
        "description": {
            "type": "String",
            "value": "Summary of this entity...",
            "metadata": {}
        },
        "temperaturetttt": {
            "type": "Number",
            "value": 23.8,
            "metadata": {}
        }
    },
    {
        "i

### Update our entity
Updating var1 value to 26.5

In [91]:
curl $ORION_SERVER:$ORION_PORT/v2/entities/$MY_UNIQUE_ID/attrs -s -S --header 'Content-Type: application/json' \
     -X PATCH -d @- <<EOF
{
  "var1": {
    "value": 26.5,
    "type": "Float"
  }
}
EOF


In [88]:
curl $ORION_SERVER:$ORION_PORT/v2/entities/$MY_UNIQUE_ID -s -S | python -m json.tool

{
    "id": "my_entityasdaxaaada",
    "type": "my_type",
    "description": {
        "type": "String",
        "value": "Summary of this entity...",
        "metadata": {}
    },
    "var1": {
        "type": "Float",
        "value": 26.5,
        "metadata": {}
    }
}



### Subscriptions

In this example we will create a subscription for $MY_ENTITY that notifies our stack about var 1 changes.

To notify changes, it will make a POST at http://$ENTITY_IP:$ENTITY_PORT passing var1, with a throttling of 5 seconds.

In [12]:
curl -v $ORION_SERVER:$ORION_PORT/v2/subscriptions -s -S --header 'Content-Type: application/json' \
    -d @- <<EOF
{
  "description": "Subscripció de so per activar la cuna",
  "subject": {
    "entities": [
      {
        "id": "bressol",
        "type": "bebe"
      }
    ],
    "condition": {
      "attrs": [
        "estat"
      ]
    }
  },
  "notification": {
    "http": {
      "url": "http://192.168.4.28"
    },
    "attrs": [
      "estat"
    ]
  },
  "expires": "2040-01-01T14:00:00.00Z",
  "throttling": 1
}
EOF


*   Trying 192.168.4.230...
* TCP_NODELAY set
* Connected to 192.168.4.230 (192.168.4.230) port 80 (#0)







* upload completely sent off: 386 out of 386 bytes







* Connection #0 to host 192.168.4.230 left intact



In [13]:
curl http://84.89.60.4/v2/entities/bressol/attrs -s -S --header 'Content-Type: application/json' \
     -X PATCH -d @- <<EOF
{
  "estat": {
    "value": "off",
    "type": "String"
  }
}
EOF